In [1]:
%cd /opt

/opt


In [2]:
%%capture
!tar xvf /kaggle/input/extract-prebuilt-kaldi-from-docker/kaldi.tar

In [3]:
%cd kaldi/egs

/opt/kaldi/egs


In [4]:
!git clone https://github.com/danijel3/ClarinStudioKaldi

Cloning into 'ClarinStudioKaldi'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 778 (delta 0), reused 0 (delta 0), pack-reused 775
Receiving objects: 100% (778/778), 35.26 MiB | 37.07 MiB/s, done.
Resolving deltas: 100% (262/262), done.


In [5]:
%cd ClarinStudioKaldi

/opt/kaldi/egs/ClarinStudioKaldi


In [6]:
#apt-get -y install libperlio-gzip-perl
!conda install -c bioconda perl-perlio-gzip -y

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [7]:
import os
#os.environ['LD_LIBRARY_PATH'] = f'{os.environ["LD_LIBRARY_PATH"]}:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'

In [8]:
!cat path.sh|sed -e 's/~\/apps/\/opt/' > tmp
!mv tmp path.sh

In [9]:
!echo > local_clarin/clarin_pl_clean.sh

In [10]:
!mkdir /kaggle/working/data
!ln -s /kaggle/working/data

In [11]:
%%writefile runmfcc.sh
#!/bin/bash

. ./path.sh ## set the paths in this file correctly!

# link to scripts from the standard Kaldi distribution
# we try to use these as much as possible
if [ ! -f $KALDI_ROOT/egs/wsj/s5/conf ] ; then ln -s $KALDI_ROOT/egs/wsj/s5/conf ; fi
if [ ! -f $KALDI_ROOT/egs/wsj/s5/local ] ; then ln -s $KALDI_ROOT/egs/wsj/s5/local ; fi
if [ ! -f $KALDI_ROOT/egs/wsj/s5/utils ] ; then ln -s $KALDI_ROOT/egs/wsj/s5/utils ; fi
if [ ! -f $KALDI_ROOT/egs/wsj/s5/steps ] ; then ln -s $KALDI_ROOT/egs/wsj/s5/steps ; fi

# exits script if error occurs anywhere
# you might not want to do this for interactive shells.
set -e

# check if the user wants to clean the project
local_clarin/clarin_pl_clean.sh

export nj=40 ##number of concurrent processes
export nj_test=30 ## number of concurrent processes for test has to be <=30

# This is a shell script, but it's recommended that you run the commands one by
# one by copying and pasting into the shell.

#run some initial data preparation (look at the file for more details):
local_clarin/clarin_pl_data_prep.sh

#prepare the lang directory
utils/prepare_lang.sh data/local/dict_nosp "<unk>" data/local/tmp_nosp data/lang_nosp

#make G.fst
utils/format_lm.sh data/lang_nosp local_clarin/arpa.lm.gz data/local/dict_nosp/lexicon.txt data/lang_nosp_test

# Make normalized MFCC features.
steps/make_mfcc.sh --nj $nj data/train
steps/compute_cmvn_stats.sh data/train
steps/make_mfcc.sh --nj $nj data/test
steps/compute_cmvn_stats.sh data/test

Writing runmfcc.sh


In [12]:
!sh runmfcc.sh

/opt/kaldi/egs/ClarinStudioKaldi/audio /opt/kaldi/egs/ClarinStudioKaldi
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4704M  100 4704M    0     0  21.3M      0  0:03:40  0:03:40 --:--:-- 21.6M
Extracting files...
/opt/kaldi/egs/ClarinStudioKaldi
Generating file lists using proper paths...
Generating spk2utt...
Preparing dictionary...
local_clarin/clarin_prepare_dict.sh data/local/all.wlist data/local/dict_nosp
local_clarin/clarin_prepare_dict.sh: line 72: phonetisaurus-apply: command not found
Dictionary preparation succeeded
utils/prepare_lang.sh data/local/dict_nosp <unk> data/local/tmp_nosp data/lang_nosp
Checking data/local/dict_nosp/silence_phones.txt ...
--> reading data/local/dict_nosp/silence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> data/local/dict_nosp/silence_phones.txt is OK

Checking da